# Predecir un parámetro en función otras variables

Se intentará predecir la Tonalidad del sonido en función de otras variables. Definiendo variables dicotómicas para aquellas variables categóricas que así lo requieran.

In [42]:
import pandas as pd
import numpy as np

from sklearn import linear_model
import statsmodels.api as sm

np.set_printoptions(precision=2)

In [43]:
data = pd.read_csv('snd-dataset-from-plain-json.csv')
data.head()

,Duration,Loudness,LogAttackTime,Tempo,Tempo.confidence,TemporalCentroid,SingleEvent,Loop,Tonality,Tonality.confidence,DynamicRange,Note.midi,Note.frequency,Note.confidence,Genre,Mood
0,24.218412,-16.581459,0.769376,95,0.133154,0.498596,False,False,G major,0.524679,9.689243,55,197.972900,0.000000,Genre B,Mood B
1,243.983673,-16.891335,1.618665,65,0.545527,0.479576,False,False,G major,0.785114,5.247044,40,85.456451,0.000000,Genre A,Mood A
2,15.281632,-21.658251,0.582658,63,0.996905,0.492315,True,True,C minor,0.698095,1.060242,50,151.972198,0.352345,Genre B,Mood B
3,2.000000,-10.525232,-1.590209,119,0.000000,0.468918,False,False,G# minor,0.646680,0.000000,41,91.402817,0.000000,Genre A,Mood A
4,1.454150,-28.335722,-0.492548,152,0.000000,0.502481,True,False,F# minor,0.408481,0.000000,107,3984.657227,0.695633,Genre A,Mood A


In [44]:
data.shape[0]

1017

In [45]:
# Se descartan columnas que tienen que ver con la confianza
# en las estimaciones en el cálculo de features
# Tip: axis number (0 for rows and 1 for columns)
data = data.drop("Tempo.confidence", axis=1);
data = data.drop("Tonality.confidence", axis=1);
data = data.drop("Note.confidence", axis=1);

# Según la documentación (Referencia: https://github.com/AudioCommons/ac-audio-extractor) Genre y Mood
# todavía no están correctamente implementadas, así que se descarta su valor
data = data.drop("Genre", axis=1);
data = data.drop("Mood", axis=1);

### Correlación entre variables

In [46]:
data.corr(method='pearson', min_periods=1) # pearson -> método estándar

,Duration,Loudness,LogAttackTime,Tempo,TemporalCentroid,SingleEvent,Loop,DynamicRange,Note.midi,Note.frequency
Duration,1.000000,0.081523,0.501519,0.104969,0.258024,-0.232345,-0.233539,0.318640,-0.154564,-0.133548
Loudness,0.081523,1.000000,0.060905,0.035220,0.046486,-0.465222,-0.070003,-0.118577,-0.105028,-0.122475
LogAttackTime,0.501519,0.060905,1.000000,0.071875,0.340716,-0.238297,-0.234266,0.359254,-0.054808,-0.100731
Tempo,0.104969,0.035220,0.071875,1.000000,0.072140,-0.021407,-0.159896,0.046447,0.012247,0.034907
TemporalCentroid,0.258024,0.046486,0.340716,0.072140,1.000000,-0.187250,-0.136633,0.038687,-0.007034,-0.021946
SingleEvent,-0.232345,-0.465222,-0.238297,-0.021407,-0.187250,1.000000,0.163248,-0.118497,0.128890,0.155892
Loop,-0.233539,-0.070003,-0.234266,-0.159896,-0.136633,0.163248,1.000000,-0.088077,0.047451,0.078194
DynamicRange,0.318640,-0.118577,0.359254,0.046447,0.038687,-0.118497,-0.088077,1.000000,0.093423,0.002418
Note.midi,-0.154564,-0.105028,-0.054808,0.012247,-0.007034,0.128890,0.047451,0.093423,1.000000,0.801161
Note.frequency,-0.133548,-0.122475,-0.100731,0.034907,-0.021946,0.155892,0.078194,0.002418,0.801161,1.000000


**Observación: La correlación entre variables en general es baja, salvo en duración con LogAttackTime, DynamicRange y TemporalCentroid en menor medida.**

## Regresión múltiple (varias variables)

In [47]:
# Referencia: https://stackoverflow.com/questions/11479064/multiple-linear-regression-in-python
def reg_multiple(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results

In [49]:
Duration = np.asarray( data.loc[:, 'Duration' ] )
DynamicRange = np.asarray( data.loc[:, 'DynamicRange' ] )
TemporalCentroid = np.asarray( data.loc[:, 'TemporalCentroid' ] )
LogAttackTime = np.asarray( data.loc[:, 'LogAttackTime' ] )

In [50]:
y = LogAttackTime
X = np.array( [Duration, DynamicRange])

reg_multiple(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.296
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     213.0
Date:                Wed, 19 Dec 2018   Prob (F-statistic):           6.06e-78
Time:                        19:22:42   Log-Likelihood:                -1329.5
No. Observations:                1017   AIC:                             2665.
Df Residuals:                    1014   BIC:                             2680.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0442      0.006      7.985      0.000       0.033       0.055
x2             0.0026      0.000     15.495      0.000       0.002       0.003
const          0.2564      0.047      5.403      0.000       0.163       0.349
==============================================================================
Omnibus:                      158.744   Durbin-Watson:                   2.052
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              235.732
Skew:                          -1.112   Prob(JB):                     6.48e-52
Kurtosis:                       3.788   Cond. No.                         407.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Observación: El R cuadrado no llega a 0.3, cuando se espera algo por lo menos de 0.5.**

## Se agrega el feature TemporalCentroid

In [51]:
y = LogAttackTime
X = np.array( [Duration, DynamicRange, TemporalCentroid])

reg_multiple(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     180.4
Date:                Wed, 19 Dec 2018   Prob (F-statistic):           1.04e-93
Time:                        19:22:44   Log-Likelihood:                -1290.2
No. Observations:                1017   AIC:                             2588.
Df Residuals:                    1013   BIC:                             2608.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.6500      0.404      9.027      0.000       2.857       4.443
x2             0.0464      0.005      8.715      0.000       0.036       0.057
x3             0.0022      0.000     13.205      0.000       0.002       0.003
const         -1.5264      0.203     -7.530      0.000      -1.924      -1.129
==============================================================================
Omnibus:                      163.368   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              247.697
Skew:                          -1.111   Prob(JB):                     1.63e-54
Kurtosis:                       3.951   Cond. No.                     4.00e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Observación: El valor de R cuadrado mejora, pero no se alcanza un valor de p > 0.005 que defina un 95% de confianza.**

## Variables categóricas  y conversión a dummy dicotómicas


**La asignación de un número a cada categoría no resuelve el problema. La solución es crear tantas variables dicotómicas como número de respuestas. Para k valores posibles, se toman k-1 variables dicotómicas.**

### Key o Tonality

Las categorias posibles para la tonalidad se encuentran en notación americana, donde 'A' es La, 'B' es Sí, etc. Se expresan en el array 'key_to_number_list' y luego se mapean en 23 variables dicotómicas, ya que la cantidad de valores posibles es 24.

### Genre, Loop y Mood

Género, si es loopeable y Mood (humor) se mapean en una única variable dicotómica ya que poseen solo dos valores posibles.

Según la documentación (https://github.com/AudioCommons/ac-audio-extractor) Genre y Mood
todavía no están correctamente implementadas, así que se descarta su valor y queda temporalmente deshabilitado

In [10]:
data

,Duration,Loudness,LogAttackTime,Tempo,TemporalCentroid,SingleEvent,Loop,Tonality,DynamicRange,Note.midi,Note.frequency,Genre,Mood
0,24.218412,-16.581459,0.769376,95,0.498596,False,False,G major,9.689243,55,197.972900,Genre B,Mood B
1,243.983673,-16.891335,1.618665,65,0.479576,False,False,G major,5.247044,40,85.456451,Genre A,Mood A
2,15.281632,-21.658251,0.582658,63,0.492315,True,True,C minor,1.060242,50,151.972198,Genre B,Mood B
3,2.000000,-10.525232,-1.590209,119,0.468918,False,False,G# minor,0.000000,41,91.402817,Genre A,Mood A
4,1.454150,-28.335722,-0.492548,152,0.502481,True,False,F# minor,0.000000,107,3984.657227,Genre A,Mood A
5,267.807343,-10.931602,1.392732,120,0.520935,False,False,C major,6.640100,49,143.887039,Genre B,Mood B
6,88.000000,-23.165480,1.241127,117,0.458141,False,False,A# major,9.204067,57,220.011536,Genre A,Mood A
7,165.276733,-11.126256,1.022743,129,0.508618,False,False,D major,3.703292,45,110.599480,Genre B,Mood B
8,3.813878,-18.993910,0.283296,108,0.645547,False,False,D minor,0.593395,61,293.014404,Genre A,Mood A
9,38.000000,-25.768625,0.658518,138,0.537336,False,False,G major,6.336580,44,108.097351,Genre B,Mood B


In [11]:
# Para mapear el 'key' en cifrado americano a numeros enteros (relación lineal entre semi-tonos)
# La tonalidad (mayor/menor) se mapea por separado)
key_to_number_list = ['A','A#','B','C','C#','D','D#','E','F','F#','G','G#']

def keyToNumber(x_value):
    key, tonality = x_value.split(' ') # [key_cifrado_americano], [major/minor]
    return [i for i,x in enumerate(key_to_number_list) if x == key][0]

In [12]:
# Cantidad de dicotómicas
len(key_to_number_list)

12

In [56]:
# Key
# Mapeo de las key/tonality 'en texto' a categorias numéricas
# Se necesitan k-1 variables dicotómicas, con k=len(key_to_number_list)
# Se separa en la key (A, A#, B, etc) y la tonalidad (mayor o menor)
# La primera utiliza 11 dicotómicas y la segunda se puede representar con una sola
#    (ya que tiene solo dos valores posibles)
data['D1'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==1 else 0)
data['D2'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==2 else 0)
data['D3'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==3 else 0)
data['D4'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==4 else 0)
data['D5'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==5 else 0)
data['D6'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==6 else 0)
data['D7'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==7 else 0)
data['D8'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==8 else 0)
data['D9'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==9 else 0)
data['D10'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==10 else 0)
data['D11'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==11 else 0)

data['T1'] = data['Tonality'].map(lambda x: 1 if x.split(' ')[1]=='major' else 0)

# Genre
#data['G1'] = data['Genre'].map(lambda x: 1 if x=='Genre A' else 0)

# Loopeable?
data['L1'] = data['Loop'].map(lambda x: 1 if x==True else 0)

# Mood
#data['M1'] = data['Mood'].map(lambda x: 1 if x=='Mood A' else 0)

data.head()

,Duration,Loudness,LogAttackTime,Tempo,TemporalCentroid,SingleEvent,Loop,Tonality,DynamicRange,Note.midi,...,D4,D5,D6,D7,D8,D9,D10,D11,T1,L1
0,24.218412,-16.581459,0.769376,95,0.498596,False,False,G major,9.689243,55,...,0,0,0,0,0,0,1,0,1,0
1,243.983673,-16.891335,1.618665,65,0.479576,False,False,G major,5.247044,40,...,0,0,0,0,0,0,1,0,1,0
2,15.281632,-21.658251,0.582658,63,0.492315,True,True,C minor,1.060242,50,...,0,0,0,0,0,0,0,0,0,1
3,2.000000,-10.525232,-1.590209,119,0.468918,False,False,G# minor,0.000000,41,...,0,0,0,0,0,0,0,1,0,0
4,1.454150,-28.335722,-0.492548,152,0.502481,True,False,F# minor,0.000000,107,...,0,0,0,0,0,1,0,0,0,0


In [57]:
L1 = np.asarray( data.loc[:, 'L1' ] )

# Genre y Mood todavía no están correctamente implementadas (ver documentación), así que se descarta su valor
# y queda temporalmente deshabilitado
#G1 = np.asarray( data.loc[:, 'G1' ] )
#M1 = np.asarray( data.loc[:, 'M1' ] )

D1 = np.asarray( data.loc[:, 'D1' ] )
D2 = np.asarray( data.loc[:, 'D2' ] )
D3 = np.asarray( data.loc[:, 'D3' ] )
D4 = np.asarray( data.loc[:, 'D4' ] )
D5 = np.asarray( data.loc[:, 'D5' ] )
D6 = np.asarray( data.loc[:, 'D6' ] )
D7 = np.asarray( data.loc[:, 'D7' ] )
D8 = np.asarray( data.loc[:, 'D8' ] )
D9 = np.asarray( data.loc[:, 'D9' ] )
D10 = np.asarray( data.loc[:, 'D10' ] )
D11 = np.asarray( data.loc[:, 'D11' ] )

T1 = np.asarray( data.loc[:, 'T1' ] )

In [58]:
# Regresión mutivariable con dicotómicas

y = LogAttackTime
#X = np.array( [Duration, DynamicRange, TemporalCentroid, G1, L1, M1, D1, D2, D3, D4, D5, D6,D7,D8,D9,D10,D11,T1])
X = np.array( [Duration, DynamicRange, TemporalCentroid, L1, T1, D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11])

reg_multiple(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     38.64
Date:                Wed, 19 Dec 2018   Prob (F-statistic):           5.86e-93
Time:                        19:25:10   Log-Likelihood:                -1263.1
No. Observations:                1017   AIC:                             2560.
Df Residuals:                    1000   BIC:                             2644.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.6162      0.146     -4.234      0.000      -0.902      -0.331
x2            -0.0139      0.105     -0.132      0.895      -0.220       0.193
x3             0.0059      0.143      0.041      0.967      -0.275       0.287
x4            -0.3064      0.131     -2.341      0.019      -0.563      -0.050
x5            -0.2025      0.103     -1.965      0.050      -0.405      -0.000
x6            -0.5211      0.165     -3.151      0.002      -0.846      -0.197
x7            -0.0951      0.106     -0.899      0.369      -0.303       0.113
x8            -0.1082      0.177     -0.611      0.541      -0.456       0.239
x9            -0.0449      0.107     -0.421      0.674      -0.254       0.165
x10           -0.1979      0.131     -1.510      0.131      -0.455       0.059
x11           -0.2368      0.133     -1.782      0.075      -0.498       0.024
x12            0.0870      0.056      1.554      0.121      -0.023       0.197
x13           -0.3575      0.098     -3.635      0.000      -0.551      -0.165
x14            3.4746      0.400      8.680      0.000       2.689       4.260
x15            0.0466      0.005      8.843      0.000       0.036       0.057
x16            0.0020      0.000     11.871      0.000       0.002       0.002
const         -1.2773      0.215     -5.943      0.000      -1.699      -0.856
==============================================================================
Omnibus:                      159.021   Durbin-Watson:                   2.054
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              241.958
Skew:                          -1.072   Prob(JB):                     2.88e-53
Kurtosis:                       4.056   Cond. No.                     4.07e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.07e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Observación: El valor de R cuadrado (coeficiente de determinación) mejora.**

## Otra estrategia: limpiar un poco más el dataset

In [59]:
data.shape[0]

1017

### Se filtran los 'single events' ya que se asume que no son canciones

Es decir 'eventos únicos', no van a caracterizar bien tonalidad, género, etc.


In [60]:
data = data[~data['SingleEvent'].isin([True])] 
data = data.drop("SingleEvent", axis=1);

data.shape[0]

871

### Se mantienen solo los sonidos de más de 120 segundos (canciones )

Criterio: Se filtran los sonidos de más de 2 minutos y menos de 5, para restringir el dataset a canciones más convencionales (menos experimentales, muy cortas o muy largas).

In [61]:
data = data[ data['Duration'] > 60*2 ]
data = data[ data['Duration'] < 60*5 ]
data.shape[0]

399

Queda un total de casi 400 instancias

## Correlación

In [62]:
data.corr(method='pearson', min_periods=1) # pearson -> método estándar

,Duration,Loudness,LogAttackTime,Tempo,TemporalCentroid,Loop,DynamicRange,Note.midi,Note.frequency,D1,...,D4,D5,D6,D7,D8,D9,D10,D11,T1,L1
Duration,1.000000,0.138401,0.078643,0.073934,-0.052267,NaN,-0.137162,-0.136723,-0.092955,-0.095537,...,-0.021973,0.052853,-0.022756,0.062770,0.069183,0.041269,-0.013114,-0.003025,-0.040923,NaN
Loudness,0.138401,1.000000,-0.035528,0.000409,-0.084362,NaN,-0.385033,-0.140231,-0.211257,-0.068300,...,0.089250,-0.004584,-0.064297,0.076069,0.037078,0.063008,-0.021518,-0.058731,0.029375,NaN
LogAttackTime,0.078643,-0.035528,1.000000,0.025246,0.196094,NaN,0.184478,-0.134777,-0.128704,-0.111130,...,0.033036,0.025346,0.022829,-0.023403,0.017756,-0.002290,0.040939,-0.100108,0.042504,NaN
Tempo,0.073934,0.000409,0.025246,1.000000,0.034338,NaN,0.080032,0.064543,-0.013254,-0.035142,...,0.122586,0.084164,0.134019,-0.073791,0.014462,-0.053636,0.017622,0.010571,0.034097,NaN
TemporalCentroid,-0.052267,-0.084362,0.196094,0.034338,1.000000,NaN,0.101423,-0.000020,-0.033961,-0.116078,...,0.020961,-0.016378,0.151150,0.047303,-0.025000,-0.024908,-0.002158,0.072472,0.046326,NaN
Loop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DynamicRange,-0.137162,-0.385033,0.184478,0.080032,0.101423,NaN,1.000000,0.195923,0.091224,0.111339,...,0.016609,-0.051923,0.026341,-0.061919,0.005620,-0.107705,-0.021539,-0.029221,0.035657,NaN
Note.midi,-0.136723,-0.140231,-0.134777,0.064543,-0.000020,NaN,0.195923,1.000000,0.819797,0.145437,...,0.001431,-0.076440,0.137204,-0.097094,0.011968,0.090194,-0.042814,0.027604,0.041740,NaN
Note.frequency,-0.092955,-0.211257,-0.128704,-0.013254,-0.033961,NaN,0.091224,0.819797,1.000000,0.120906,...,-0.015301,-0.031122,0.055945,-0.073325,-0.001087,0.131475,-0.000113,-0.001857,0.058266,NaN
D1,-0.095537,-0.068300,-0.111130,-0.035142,-0.116078,NaN,0.111339,0.145437,0.120906,1.000000,...,-0.038679,-0.091854,-0.038679,-0.093782,-0.045402,-0.046952,-0.104120,-0.040451,0.047819,NaN


**Observación: La correlación baja.**

## Otra estrategia: Analizar los single events

In [63]:
data = pd.read_csv('snd-dataset-from-plain-json.csv')
data = data[data['SingleEvent'].isin([True])] 

data.shape[0]

146

In [64]:
# eventos cortos
data = data[ data['Duration'] < 5 ]
data.shape[0]

29

In [65]:
data.corr(method='pearson', min_periods=1) # pearson -> método estándar

,Duration,Loudness,LogAttackTime,Tempo,Tempo.confidence,TemporalCentroid,SingleEvent,Loop,Tonality.confidence,DynamicRange,Note.midi,Note.frequency,Note.confidence
Duration,1.000000,0.409527,0.458463,0.413927,0.109870,-0.003603,NaN,0.111727,0.267255,0.314945,-0.140897,-0.048192,0.070128
Loudness,0.409527,1.000000,0.093151,0.011048,0.254183,-0.259784,NaN,0.254445,0.122818,0.344705,-0.113036,-0.263562,0.137129
LogAttackTime,0.458463,0.093151,1.000000,0.299679,-0.105107,0.646900,NaN,-0.106338,0.292855,-0.274957,0.360239,0.351031,0.465610
Tempo,0.413927,0.011048,0.299679,1.000000,-0.049662,0.220953,NaN,-0.049214,0.106185,0.006667,0.195123,0.199756,0.195006
Tempo.confidence,0.109870,0.254183,-0.105107,-0.049662,1.000000,-0.096474,NaN,0.999910,-0.144554,0.173904,-0.070938,-0.152904,-0.010962
TemporalCentroid,-0.003603,-0.259784,0.646900,0.220953,-0.096474,1.000000,NaN,-0.102161,0.340015,-0.656999,0.343358,0.257487,0.368906
SingleEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loop,0.111727,0.254445,-0.106338,-0.049214,0.999910,-0.102161,NaN,1.000000,-0.145321,0.180476,-0.070443,-0.152925,-0.012989
Tonality.confidence,0.267255,0.122818,0.292855,0.106185,-0.144554,0.340015,NaN,-0.145321,1.000000,-0.285782,-0.502435,-0.548159,0.184913
DynamicRange,0.314945,0.344705,-0.274957,0.006667,0.173904,-0.656999,NaN,0.180476,-0.285782,1.000000,-0.121501,-0.129814,-0.149882


**Observación:** Se observa que la variable LogAttackTime esta bastante correlacionada linealmente con Duration, TemporaCentroid, por lo cuál se espera una regresión aceptable.

In [66]:
Duration = np.asarray( data.loc[:, 'Duration' ] )
DynamicRange = np.asarray( data.loc[:, 'DynamicRange' ] )
TemporalCentroid = np.asarray( data.loc[:, 'TemporalCentroid' ] )
LogAttackTime = np.asarray( data.loc[:, 'LogAttackTime' ] )
#Tempo = np.asarray( data.loc[:, 'Tempo' ] )

y = LogAttackTime
X = np.array( [Duration, TemporalCentroid])

reg_multiple(y, X).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     22.21
Date:                Wed, 19 Dec 2018   Prob (F-statistic):           2.37e-06
Time:                        19:25:39   Log-Likelihood:                -19.970
No. Observations:                  29   AIC:                             45.94
Df Residuals:                      26   BIC:                             50.04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.6067      0.663      5.443      0.000       2.245       4.969
x2             0.2551      0.066      3.867      0.001       0.119       0.391
const         -2.9599      0.336     -8.805      0.000      -3.651      -2.269
==============================================================================
Omnibus:                        3.600   Durbin-Watson:                   1.777
Prob(Omnibus):                  0.165   Jarque-Bera (JB):                3.174
Skew:                          -0.786   Prob(JB):                        0.205
Kurtosis:                       2.604   Cond. No.                         22.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Se obtiene un valor aceptable de R cuadrado, pero no así para p**

## Filtrar solo los features calculados con confianza alta

In [67]:
data = pd.read_csv('snd-dataset-from-plain-json.csv')

#data = data[ data['Tempo.confidence'] > 0.5 ]
data = data[ data['Tonality.confidence'] > 0.8 ]
data = data[ data['Note.confidence'] > 0.5 ]

data.shape[0]

32

In [68]:
data.corr(method='pearson', min_periods=1) # pearson -> método estándar

,Duration,Loudness,LogAttackTime,Tempo,Tempo.confidence,TemporalCentroid,SingleEvent,Loop,Tonality.confidence,DynamicRange,Note.midi,Note.frequency,Note.confidence
Duration,1.000000,0.304569,0.642006,0.242524,0.034554,0.089504,NaN,-0.082728,0.442781,0.356598,0.067216,-0.023755,-0.234207
Loudness,0.304569,1.000000,-0.079180,0.032689,-0.122179,0.044776,NaN,-0.168705,0.144841,-0.310329,0.152639,0.116591,-0.320799
LogAttackTime,0.642006,-0.079180,1.000000,0.067663,-0.020464,0.264089,NaN,0.165728,0.243919,0.513857,-0.023101,-0.142450,-0.225773
Tempo,0.242524,0.032689,0.067663,1.000000,-0.510652,0.338942,NaN,-0.309448,0.231979,-0.068813,-0.075381,-0.055689,-0.125349
Tempo.confidence,0.034554,-0.122179,-0.020464,-0.510652,1.000000,-0.281462,NaN,0.774390,-0.058799,0.186137,-0.069464,-0.061536,0.280368
TemporalCentroid,0.089504,0.044776,0.264089,0.338942,-0.281462,1.000000,NaN,-0.069671,0.098502,-0.264802,0.258988,0.158615,0.018859
SingleEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loop,-0.082728,-0.168705,0.165728,-0.309448,0.774390,-0.069671,NaN,1.000000,-0.205802,0.084184,-0.125787,-0.147280,0.264528
Tonality.confidence,0.442781,0.144841,0.243919,0.231979,-0.058799,0.098502,NaN,-0.205802,1.000000,0.163412,0.062781,0.090452,0.062282
DynamicRange,0.356598,-0.310329,0.513857,-0.068813,0.186137,-0.264802,NaN,0.084184,0.163412,1.000000,-0.003761,-0.004028,-0.211804


In [70]:
data['L1'] = data['Loop'].map(lambda x: 1 if x==True else 0)

# Genre y Mood deshabilitadas por ahora (ver doc)
#data['G1'] = data['Genre'].map(lambda x: 1 if x=='Genre A' else 0)
#data['M1'] = data['Mood'].map(lambda x: 1 if x=='Mood A' else 0)

# Tonality / Key
# Mapeo de las tonalidades 'en texto' a categorias numéricas
# Se necesitan k-1 variables dicotómicas, con k=len(key_to_number_list)
data['D1'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==1 else 0)
data['D2'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==2 else 0)
data['D3'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==3 else 0)
data['D4'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==4 else 0)
data['D5'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==5 else 0)
data['D6'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==6 else 0)
data['D7'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==7 else 0)
data['D8'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==8 else 0)
data['D9'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==9 else 0)
data['D10'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==10 else 0)
data['D11'] = data['Tonality'].map(lambda x: 1 if keyToNumber(x)==11 else 0)

data['T1'] = data['Tonality'].map(lambda x: 1 if x.split(' ')[1]=='major' else 0)

#data.head()

#G1 = np.asarray( data.loc[:, 'G1' ] ) # Genre
L1 = np.asarray( data.loc[:, 'L1' ] ) # Loop
#M1 = np.asarray( data.loc[:, 'M1' ] ) # Mood

D1 = np.asarray( data.loc[:, 'D1' ] )
D2 = np.asarray( data.loc[:, 'D2' ] )
D3 = np.asarray( data.loc[:, 'D3' ] )
D4 = np.asarray( data.loc[:, 'D4' ] )
D5 = np.asarray( data.loc[:, 'D5' ] )
D6 = np.asarray( data.loc[:, 'D6' ] )
D7 = np.asarray( data.loc[:, 'D7' ] )
D8 = np.asarray( data.loc[:, 'D8' ] )
D9 = np.asarray( data.loc[:, 'D9' ] )
D10 = np.asarray( data.loc[:, 'D10' ] )
D11 = np.asarray( data.loc[:, 'D11' ] )

T1 = np.asarray( data.loc[:, 'T1' ] )

In [71]:
Duration = np.asarray( data.loc[:, 'Duration' ] )
DynamicRange = np.asarray( data.loc[:, 'DynamicRange' ] )
LogAttackTime = np.asarray( data.loc[:, 'LogAttackTime' ] )
Loudness = np.asarray( data.loc[:, 'Loudness' ] )
Tempo = np.asarray( data.loc[:, 'Tempo' ] )
TemporalCentroid = np.asarray( data.loc[:, 'TemporalCentroid' ] )

y = LogAttackTime
#X = np.array( [Loudness, TemporalCentroid, Duration] )
# X = np.array( [DynamicRange, TemporalCentroid, Duration, G1, L1, M1, D1, D2, D3, D4, D5, D6,D7,D8,D9,D10,D11,T1])
# Según la documentación (Referencia: https://github.com/AudioCommons/ac-audio-extractor) Genre y Mood
# todavía no están correctamente implementadas, así que se descarta su valor
X = np.array( [DynamicRange, TemporalCentroid, Duration, L1, D1, D2, D3, D4, D5, D6,D7,D8,D9,D10,D11,T1])


reg_multiple(y, X).summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.666
Method:                 Least Squares   F-statistic:                     7.186
Date:                Wed, 19 Dec 2018   Prob (F-statistic):           7.76e-05
Time:                        19:26:27   Log-Likelihood:                -18.221
No. Observations:                  32   AIC:                             58.44
Df Residuals:                      21   BIC:                             74.57
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.1131      0.206     -0.548      0.589      -0.542       0.316
const       1.765e-14   5.43e-15      3.252      0.004    6.36e-15    2.89e-14
x2            -0.7775      0.445     -1.746      0.095      -1.703       0.148
x3         -6.159e-16   2.47e-16     -2.498      0.021   -1.13e-15   -1.03e-16
x4            -1.5105      0.472     -3.203      0.004      -2.491      -0.530
x5            -0.6144      0.441     -1.395      0.178      -1.531       0.302
x6          3.505e-16   3.52e-16      0.996      0.331   -3.82e-16    1.08e-15
x7            -0.9578      0.384     -2.493      0.021      -1.757      -0.159
x8         -2.312e-15   7.98e-16     -2.897      0.009   -3.97e-15   -6.52e-16
x9            -0.5474      0.378     -1.447      0.163      -1.334       0.240
x10          -1.8e-15   7.04e-16     -2.557      0.018   -3.26e-15   -3.36e-16
x11                 0          0        nan        nan           0           0
x12            0.5264      0.349      1.510      0.146      -0.198       1.251
x13            0.0039      0.002      2.552      0.019       0.001       0.007
x14            5.1382      1.593      3.226      0.004       1.826       8.450
x15            0.0658      0.028      2.359      0.028       0.008       0.124
x16           -1.6662      0.889     -1.873      0.075      -3.516       0.184
==============================================================================
Omnibus:                        5.940   Durbin-Watson:                   1.777
Prob(Omnibus):                  0.051   Jarque-Bera (JB):                4.285
Skew:                          -0.793   Prob(JB):                        0.117
Kurtosis:                       3.836   Cond. No.                     1.52e+33
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.31e-61. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

**Observación: Al filtrar el dataset solo utilizando las instancias que tenian los features calculados solo los features calculados con confianza alta es que se obtuvo los mejores resultados. R cuadrado (ajustado) de 0.666 y buenos valores para p.**


---

**Siguiente:** [3 - Reducción de la dimensionalidad SVD y PCA](3%20-%20Reducción%20de%20la%20dimensionalidad%20SVD%20y%20PCA.ipynb)